# Requirements

## Environment

1. Anaconda Python distribution.
2. Jupyter server (see _extras/jupyter_server.sh_ for details).
3. Anaconda environments (run _setup_conda_envs.sh_).
4. gmsh (not necessary if you already have meshes in either MSH or XDMF format).


## Setup

### Geometry: mesh

You need to have a mesh in XDMF format.  Try:

    conda activate kesi37
    cd extras/FEM/_meshes_new
    snakemake finite_slice_fine.xdmf -j 1
    
It may take a while.  Sadly, errors are expected.  If the build fails due to an error in the _MSH_ rule, you may need to remove the intermediate _finite_slice_fine.msh_ file and remove `-optimize_netgen` _gmsh_ flag from _Snakefile_ to disable mesh optimization which may crash _gmsh_ with a segmentation fault.

Now you have the following files created:
- _finite\_slice\_fine.xdmf_,
- _finite\_slice\_fine.h5_,
- _finite\_slice\_fine_boundaries.xdmf_,
- _finite\_slice\_fine_boundaries.h5_,
- _finite\_slice\_fine_subdomains.xdmf_,
- _finite\_slice\_fine_subdomains.h5_
(_*.xdmf_ files are headers for _*.h5_ files).  The files are derived from _finite\_slice\_fine.msh_ mesh saved in _gmsh_ format, which is created from blueprints in the _finite\_slice\_fine.geo_.  The blueprint itself is derived from a template _finite\_slice\_fine.geo.template_.  

The template may also derive meshes of other resolutions:
- _finite\_slice\_finest.xdmf_,
- _finite\_slice\_finer.xdmf_,
- _finite\_slice.xdmf_,
- _finite\_slice\_coarse.xdmf_,
- _finite\_slice\_coarser.xdmf_, and
- _finite\_slice\_coarsest.xdmf_.

There are also blueprints for _finite_slice_composite.xdmf_.

There are also templates for single sphere geometries:
- _one\_sphere\_composite.geo.template_,
- _one\_sphere\_plain.geo.template_, and
- _one\_sphere\_uniform\_cortex.geo.template_.
The recommended meshes derived from them are:
- _one\_sphere\_composite\_finest.xdmf_,
- _one\_sphere\_plain\_finer.xdmf_, 
- _one\_sphere\_plain\_finest.xdmf_, 
- _one\_sphere\_uniform\_cortex\_fine.xdmf_,
- _one\_sphere\_uniform\_cortex\_finer.xdmf_, and
- _one\_sphere\_uniform\_cortex\_finest.xdmf_.

For four sphere geometry there are blueprints from which you can derive:
- _four\_spheres\_separate\_cortex\_composite.xdmf_,
- _four\_spheres\_separate\_cortex\_composite\_fine.xdmf_, and
- _four\_spheres\_plain.xdmf_.

### Geometry: properties

For every mesh additional information is necessary, like conductivity of its compartments.  Such information is stored in the following files:
- _extras/FEM/\_meshes\_new/finite\_slice.ini_,
- _extras/FEM/\_meshes\_new/one\_sphere.ini_, and
- _extras/FEM/\_meshes\_new/four\_spheres.ini_.

Format of such file is:

    [<compartment name>]
    volume = <volume number>
    conductivity = <conductivity in SI units>
    
for a compartment and:

    [<surface name>]
    surface = <surface number>
    
for a boundary.  Additional information may be provided, like radius, thickness, or conductivity associated with external surface for subtraction method.

### FEM

In the _extras/FEM/fem\_configs_ you can find examplary configurations of FEM solver in the _fem_ section of INI files:

    [fem]
    mesh = _meshes_new/finite_slice_composite.xdmf
    config = _meshes_new/finite_slice.ini
    element_type = CG
    degree = 3
    solution_metadata_filename = solutions/paper/finite_slice/composite_3/grid_3d.ini

`mesh` is a path to the mesh header, while `config` - to properties of its compartments and boundaries.  `element_type` (`CG` for _continuous Galerkin_) and `degree` is configuration of elements used by FEM.
`solution_metadata_filename` points to a directory where the solutions are to be stored, and an INI file containing their metadata.


### electrodes

Other sections of the INI files have format:

    [<electrode name>]
    filename = <solution filename.h5>
    x = <X coordinate in meters>
    y = <Y coordinate in meters>
    z = <Z coordinate in meters>
    
and define name and spatial location of electrodes.  They also define the (relative) path to the file where the leadfield correction is to be stored.

# Preprocessing

Lets define a simple setup of three point electrodes:

    [fem]
    mesh = _meshes_new/finite_slice_fine.xdmf
    config = _meshes_new/finite_slice.ini
    element_type = CG
    degree = 3
    solution_metadata_filename = solutions/tutorial/finite_slice/fine_3/demo.ini
    
    [first]
    filename = demo/first.h5
    x = 0
    y = 0
    z = 0.5e-4
    
    [second]
    filename = demo/second.h5
    x = 0.5e-4
    y = 0
    z = 1.5e-4
    
    [third]
    filename = demo/third.h5
    x = 0.5e-4
    y = -0.5e-4
    z = 2.5e-4
    
Write the setup as _extras/FEM/fem\_configs/tutorial/finite\_slice/fine\_3/demo.ini_.
 
For every electrode solve for the leadfield correction:

    cd extras
    python paper_solve_slice_on_plate.py -c FEM/fem_configs/tutorial/finite_slice/fine_3/demo.ini -n first -o FEM/solutions/tutorial/finite_slice/fine_3/demo/first.ini
    python paper_solve_slice_on_plate.py -c FEM/fem_configs/tutorial/finite_slice/fine_3/demo.ini -n second -o FEM/solutions/tutorial/finite_slice/fine_3/demo/second.ini
    python paper_solve_slice_on_plate.py -c FEM/fem_configs/tutorial/finite_slice/fine_3/demo.ini -n third -o FEM/solutions/tutorial/finite_slice/fine_3/demo/third.ini
    
Then join the scattered metadata.

In [ ]:
import configparser

metadata = configparser.ConfigParser()
for filename in ['FEM/solutions/tutorial/finite_slice/fine_3/demo/first.ini',
                 'FEM/solutions/tutorial/finite_slice/fine_3/demo/second.ini',
                 'FEM/solutions/tutorial/finite_slice/fine_3/demo/third.ini',
                 ]:
    metadata.read(filename)
    metadata.write(open('FEM/solutions/tutorial/finite_slice/fine_3/demo.ini', 'w'))